# Drug Annotation

In [1]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set()
import os,sys
from scipy import stats
from collections import Counter
plt.style.use('seaborn-white')

## GDSC1

In [3]:
gdsc1_cid=pd.read_csv('../data/GDSC1_drug_cid_table_curated.csv')
gdsc1_info_sel=gdsc1_cid[~gdsc1_cid['pubchem'].isin(['-','None','none'])]

## GDSC2

In [4]:
gdsc2_cid=pd.read_csv('../data/GDSC2_drug_cid_table_curated.csv')
gdsc2_cid['Drug_name_id']=['GDSC2_'+x+'_'+str(y) for x,y in zip(gdsc2_cid['drug_name'],gdsc2_cid['drug_id'])]
gdsc2_info_sel=gdsc2_cid[~gdsc2_cid['pubchem'].isin(['-','None','none'])]

## CTRP2

In [7]:
ctrp2_info=pd.read_csv('../download/CTRPv2.0._INFORMER_SET.csv')
ctrp2_info_sel=ctrp2_info[ctrp2_info['cpd_status'].isin(['clinical','FDA'])]
ctrp2_info_sel=ctrp2_info_sel[~ctrp2_info_sel['master_cpd_id'].isna()]
dict_cpdid_name_ctrp2={k:v for k,v in zip(ctrp2_info_sel['master_cpd_id'],ctrp2_info_sel['cpd_name'])}

In [9]:
ctrp2_anno=pd.read_csv('../data/CTRP2_pharmacoDB_drug_info_curated.txt',sep='\t')
ctrp2_add=ctrp2_anno[['master_cpd_id','cid']].fillna(-1).astype(int)

In [10]:
ctrp2_info_sel=ctrp2_info_sel.merge(ctrp2_add,left_on='master_cpd_id',right_on='master_cpd_id',how='left')

## CTRP1

In [11]:
ctrp1_info=pd.read_csv('../download/CTRPv1.0._INFORMER_SET.csv')
ctrp1_info_sel=ctrp1_info[ctrp1_info['cpd_status'].isin(['clinical candidate', 'FDA-approved'])]
ctrp1_info_sel=ctrp1_info_sel[~ctrp1_info_sel['master_cpd_id'].isna()]
dict_cpdname_did_ctrp1={v:k for k,v in zip(ctrp1_info_sel['master_cpd_id'],ctrp1_info_sel['cpd_name'])}

In [12]:
ctrp1_info_sel[ctrp1_info_sel['pubchem_cid'].isna()]

,cpd_name,cpd_synonym,cpd_status,target_or_activity_of_compound,gene_symbol_of_protein_target,top_test_conc_umol,percent_cpd_purity,cpd_smiles,pubchem_cid,broad_cpd_id,master_cpd_id
308,sirolimus,rapamycin;Rapamune,FDA-approved,inhibitor of mTOR via FRB domain (in complex w...,MTOR,37.0,NaN,CO[C@@H]1C[C@H](C[C@@H](C)[C@@H]2CC(=O)[C@H](C...,NaN,BRD-A23770159,411830
316,tacrolimus,FK-506;fujimycin;Prograf;Advagraf;Protopic,FDA-approved,inhibitor of calcineurin (in complex with FKBP),PPP3CA;PPP3CB;PPP3CC;PPP3R1;PPP3R2,74.0,NaN,CO[C@@H]1CC(CC[C@H]1O)\C=C(/C)\[C@H]2OC(=O)[C@...,NaN,BRD-K08845546,55450


In [13]:
ctrp1_info_sel.loc[308,'pubchem_cid']=5284616
ctrp1_info_sel.loc[316,'pubchem_cid']=445643

## PRISM

In [15]:
prism_anno=pd.read_csv('../data/PRISM_pharmacoDB_drug_info_curated.txt',sep='\t')
prism_anno['disease.area']=prism_anno['disease.area'].fillna('NoData')

In [16]:
flag=prism_anno['disease.area'].str.contains('oncology')|prism_anno['disease.area'].str.contains('malignancy')
prism_anno=prism_anno[flag]
prism_anno=prism_anno[~prism_anno['cid'].isna()]
prism_anno['cid']=prism_anno['cid'].astype(int)                    

In [17]:
prism_anno_dup=prism_anno[prism_anno['broad_id'].str.contains('///')]
prism_anno_ndup=prism_anno[~prism_anno['broad_id'].str.contains('///')]

In [18]:
dupids=prism_anno_dup['broad_id']

In [19]:
adds=[]
for dupid in dupids:
    bids=dupid.split('///')
    add=prism_anno_dup[prism_anno_dup['broad_id']==dupid]
    add=pd.concat([add,add])
    add.loc[:,'broad_id']=bids
    adds.append(add)
prism_anno_dup_c=pd.concat(adds)
prism_anno_dup_c

,Unnamed: 0,PRISM.drugid,broad_id,compound_plate,moa,target,disease.area,indication,smiles,phase,drugid,cid,inchikey
33,Abiraterone-acetate,abiraterone-acetate,BRD-K50071428-001-03-3,PREP020,androgen biosynthesis inhibitor,"CYP11B1, CYP17A1///CYP17A1",oncology,prostate cancer,CC(=O)O[C@H]1CC[C@]2(C)[C@H]3CC[C@@]4(C)[C@@H]...,Launched,Abiraterone,9821849,GZOSMCIZMLWJML-VJLLXTKPSA-N
33,Abiraterone-acetate,abiraterone-acetate,BRD-K24048528-001-02-5,PREP020,androgen biosynthesis inhibitor,"CYP11B1, CYP17A1///CYP17A1",oncology,prostate cancer,CC(=O)O[C@H]1CC[C@]2(C)[C@H]3CC[C@@]4(C)[C@@H]...,Launched,Abiraterone,9821849,GZOSMCIZMLWJML-VJLLXTKPSA-N
54,Afatinib,afatinib,BRD-K66175015-001-12-4,PROS003_PR500///PREP021,EGFR inhibitor,"EGFR, ERBB2, ERBB4",oncology,non-small cell lung cancer (NSCLC),CN(C)C\C=C\C(=O)Nc1cc2c(Nc3ccc(F)c(Cl)c3)ncnc2...,Launched,Afatinib,10184653,ULXXDDBFHOBEHA-CWDCEQMOSA-N
54,Afatinib,afatinib,BRD-K66175015-001-09-0,PROS003_PR500///PREP021,EGFR inhibitor,"EGFR, ERBB2, ERBB4",oncology,non-small cell lung cancer (NSCLC),CN(C)C\C=C\C(=O)Nc1cc2c(Nc3ccc(F)c(Cl)c3)ncnc2...,Launched,Afatinib,10184653,ULXXDDBFHOBEHA-CWDCEQMOSA-N
59,Alectinib,alectinib,BRD-K11267252-001-04-4,PREP020///PROS003_PR500,ALK tyrosine kinase receptor inhibitor,"ALK, MET",oncology,non-small cell lung cancer (NSCLC),CCc1cc2C(=O)c3c([nH]c4cc(ccc34)C#N)C(C)(C)c2cc...,Launched,Alectinib,49806720,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1379,Vemurafenib,vemurafenib,BRD-K56343971-001-10-6,PROS004_PR500///PREP022,RAF inhibitor,"BRAF, RAF1",oncology,melanoma,CCCS(=O)(=O)Nc1ccc(F)c(C(=O)c2c[nH]c3ncc(cc23)...,Launched,Vemurafenib,42611257,GPXBXXGIAQBQNI-UHFFFAOYSA-N
1380,Venetoclax,venetoclax,BRD-K62391742-001-09-7,PROS001_PR500///PREP045,BCL inhibitor,BCL2,hematologic malignancy,chronic lymphocytic leukemia (CLL),CC1(C)CCC(CN2CCN(CC2)c2ccc(C(=O)NS(=O)(=O)c3cc...,Launched,Venetoclax,49846579,NaN
1380,Venetoclax,venetoclax,BRD-K62391742-001-03-0,PROS001_PR500///PREP045,BCL inhibitor,BCL2,hematologic malignancy,chronic lymphocytic leukemia (CLL),CC1(C)CCC(CN2CCN(CC2)c2ccc(C(=O)NS(=O)(=O)c3cc...,Launched,Venetoclax,49846579,NaN
1394,Vorinostat,vorinostat,BRD-K81418486-001-47-5,PREP048///PROS001_PR500,HDAC inhibitor,"HDAC1, HDAC10, HDAC11, HDAC2, HDAC3, HDAC5, HD...",hematologic malignancy,cutaneous T-cell lymphoma (CTCL),"ONC(=O)CCCCCCC(=O)Nc1ccccc1, ONC(=O)CCCCCCC(=O...",Launched,Vorinostat,5311,WAEXFXRVDQXREF-UHFFFAOYSA-N


In [20]:
prism_anno_r=pd.concat([prism_anno_ndup,prism_anno_dup_c])
bid_dup=prism_anno_r[prism_anno_r['broad_id'].duplicated()]['broad_id'].values[0]
prism_anno_r[prism_anno_r['broad_id']==bid_dup]

,Unnamed: 0,PRISM.drugid,broad_id,compound_plate,moa,target,disease.area,indication,smiles,phase,drugid,cid,inchikey
32,Abiraterone,abiraterone,BRD-K50071428-001-03-3,PREP031,androgen biosynthesis inhibitor,"CYP11B1, CYP17A1///CYP17A1",oncology,prostate cancer,C[C@]12CC[C@H]3[C@@H](CC=C4C[C@@H](O)CC[C@]34C...,Launched,Abiraterone,132971,GZOSMCIZMLWJML-VJLLXTKPSA-N
33,Abiraterone-acetate,abiraterone-acetate,BRD-K50071428-001-03-3,PREP020,androgen biosynthesis inhibitor,"CYP11B1, CYP17A1///CYP17A1",oncology,prostate cancer,CC(=O)O[C@H]1CC[C@]2(C)[C@H]3CC[C@@]4(C)[C@@H]...,Launched,Abiraterone,9821849,GZOSMCIZMLWJML-VJLLXTKPSA-N


In [21]:
did_dup=prism_anno_r[prism_anno_r['PRISM.drugid'].duplicated()]['PRISM.drugid']
prism_anno_r[prism_anno_r['PRISM.drugid'].isin(did_dup)]

,Unnamed: 0,PRISM.drugid,broad_id,compound_plate,moa,target,disease.area,indication,smiles,phase,drugid,cid,inchikey
33,Abiraterone-acetate,abiraterone-acetate,BRD-K50071428-001-03-3,PREP020,androgen biosynthesis inhibitor,"CYP11B1, CYP17A1///CYP17A1",oncology,prostate cancer,CC(=O)O[C@H]1CC[C@]2(C)[C@H]3CC[C@@]4(C)[C@@H]...,Launched,Abiraterone,9821849,GZOSMCIZMLWJML-VJLLXTKPSA-N
33,Abiraterone-acetate,abiraterone-acetate,BRD-K24048528-001-02-5,PREP020,androgen biosynthesis inhibitor,"CYP11B1, CYP17A1///CYP17A1",oncology,prostate cancer,CC(=O)O[C@H]1CC[C@]2(C)[C@H]3CC[C@@]4(C)[C@@H]...,Launched,Abiraterone,9821849,GZOSMCIZMLWJML-VJLLXTKPSA-N
54,Afatinib,afatinib,BRD-K66175015-001-12-4,PROS003_PR500///PREP021,EGFR inhibitor,"EGFR, ERBB2, ERBB4",oncology,non-small cell lung cancer (NSCLC),CN(C)C\C=C\C(=O)Nc1cc2c(Nc3ccc(F)c(Cl)c3)ncnc2...,Launched,Afatinib,10184653,ULXXDDBFHOBEHA-CWDCEQMOSA-N
54,Afatinib,afatinib,BRD-K66175015-001-09-0,PROS003_PR500///PREP021,EGFR inhibitor,"EGFR, ERBB2, ERBB4",oncology,non-small cell lung cancer (NSCLC),CN(C)C\C=C\C(=O)Nc1cc2c(Nc3ccc(F)c(Cl)c3)ncnc2...,Launched,Afatinib,10184653,ULXXDDBFHOBEHA-CWDCEQMOSA-N
59,Alectinib,alectinib,BRD-K11267252-001-04-4,PREP020///PROS003_PR500,ALK tyrosine kinase receptor inhibitor,"ALK, MET",oncology,non-small cell lung cancer (NSCLC),CCc1cc2C(=O)c3c([nH]c4cc(ccc34)C#N)C(C)(C)c2cc...,Launched,Alectinib,49806720,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1379,Vemurafenib,vemurafenib,BRD-K56343971-001-10-6,PROS004_PR500///PREP022,RAF inhibitor,"BRAF, RAF1",oncology,melanoma,CCCS(=O)(=O)Nc1ccc(F)c(C(=O)c2c[nH]c3ncc(cc23)...,Launched,Vemurafenib,42611257,GPXBXXGIAQBQNI-UHFFFAOYSA-N
1380,Venetoclax,venetoclax,BRD-K62391742-001-09-7,PROS001_PR500///PREP045,BCL inhibitor,BCL2,hematologic malignancy,chronic lymphocytic leukemia (CLL),CC1(C)CCC(CN2CCN(CC2)c2ccc(C(=O)NS(=O)(=O)c3cc...,Launched,Venetoclax,49846579,NaN
1380,Venetoclax,venetoclax,BRD-K62391742-001-03-0,PROS001_PR500///PREP045,BCL inhibitor,BCL2,hematologic malignancy,chronic lymphocytic leukemia (CLL),CC1(C)CCC(CN2CCN(CC2)c2ccc(C(=O)NS(=O)(=O)c3cc...,Launched,Venetoclax,49846579,NaN
1394,Vorinostat,vorinostat,BRD-K81418486-001-47-5,PREP048///PROS001_PR500,HDAC inhibitor,"HDAC1, HDAC10, HDAC11, HDAC2, HDAC3, HDAC5, HD...",hematologic malignancy,cutaneous T-cell lymphoma (CTCL),"ONC(=O)CCCCCCC(=O)Nc1ccccc1, ONC(=O)CCCCCCC(=O...",Launched,Vorinostat,5311,WAEXFXRVDQXREF-UHFFFAOYSA-N


In [22]:
prism_info_sel=prism_anno_r[~prism_anno_r['broad_id'].duplicated()]
prism_info_sel=prism_info_sel.reset_index(drop=True)

## gCSI

In [23]:
gcsi_anno=pd.read_csv('../download/gCSI2019_drug_info.txt'
                     ,sep='\t',index_col=0)
gcsi_info_sel=gcsi_anno[~gcsi_anno['cid'].isna()]
gcsi_info_sel['cid']=gcsi_info_sel['cid'].astype(int).astype(str)

/var/folders/3g/5zyn6dld5rl1rc3t1fmh0t240000gn/T/ipykernel_54523/965823009.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gcsi_info_sel['cid']=gcsi_info_sel['cid'].astype(int).astype(str)


In [24]:
cids= set(gdsc1_info_sel['pubchem'].astype(int))|set(gdsc2_info_sel['pubchem'].astype(int))|set(ctrp2_info_sel['cid'].astype(int))|\
set(prism_info_sel['cid'].astype(int))|set(gcsi_info_sel['cid'].astype(int))|set(ctrp1_info_sel['pubchem_cid'].astype(int))
cids=sorted(cids)[1:]
with open('../data/'+'All_cids.csv','w') as fw:
    fw.write( '\n'.join(list(map(str,cids)))+'\n' )

* this file is converted by Pubchem website, https://pubchem.ncbi.nlm.nih.gov/#upload=true
to "All_cids_from_pubchem.csv"

In [25]:
pubchem=pd.read_csv('../data/All_cids_converted_from_pubchem.csv')
dict_cid_name={k:v for k,v in zip(pubchem['cid'],pubchem['cmpdname'])}
pubchem['cmpdname'].duplicated().sum()

0

# Drug categorization

## GDSC

In [26]:
Counter(gdsc1_info_sel['pathway_name']).most_common()

[('Other', 32),
 ('Other, kinases', 27),
 ('RTK signaling', 26),
 ('PI3K/MTOR signaling', 23),
 ('ERK MAPK signaling', 17),
 ('DNA replication', 12),
 ('Cell cycle', 12),
 ('Mitosis', 11),
 ('Chromatin histone acetylation', 10),
 ('Protein stability and degradation', 8),
 ('Genome integrity', 8),
 ('EGFR signaling', 7),
 ('Cytoskeleton', 7),
 ('Chromatin other', 6),
 ('JNK and p38 signaling', 6),
 ('Apoptosis regulation', 6),
 ('Metabolism', 5),
 ('WNT signaling', 4),
 ('Hormone-related', 3),
 ('IGF1R signaling', 3),
 ('Chromatin histone methylation', 3),
 ('p53 pathway', 3),
 ('ABL signaling', 2)]

In [27]:
Counter(gdsc2_info_sel['pathway_name']).most_common()

[('PI3K/MTOR signaling', 21),
 ('DNA replication', 20),
 ('Other', 17),
 ('Other, kinases', 14),
 ('Genome integrity', 13),
 ('ERK MAPK signaling', 13),
 ('Apoptosis regulation', 12),
 ('RTK signaling', 11),
 ('Cell cycle', 10),
 ('WNT signaling', 9),
 ('Protein stability and degradation', 9),
 ('Mitosis', 8),
 ('EGFR signaling', 7),
 ('Chromatin histone methylation', 7),
 ('Chromatin other', 7),
 ('Hormone-related', 5),
 ('Metabolism', 4),
 ('IGF1R signaling', 4),
 ('Chromatin histone acetylation', 4),
 ('p53 pathway', 4),
 ('-', 2),
 ('JNK and p38 signaling', 2),
 ('Cytoskeleton', 2),
 ('ABL signaling', 1)]

In [28]:
Counter(gdsc1_info_sel['targets']).most_common()[:50]

[('MEK1, MEK2', 7),
 ('BRAF', 6),
 ('HSP90', 5),
 ('PARP1, PARP2', 5),
 ('AKT1, AKT2, AKT3', 4),
 ('EGFR', 4),
 ('IGF1R, IR', 4),
 ('HDAC1', 4),
 ('EGFR, ERBB2', 3),
 ('Microtubule stabiliser', 3),
 ('GSK3A, GSK3B', 3),
 ('PI3K (class 1)', 3),
 ('ROCK1, ROCK2', 3),
 ('AR', 2),
 ('Antimetabolite', 2),
 ('dsDNA break induction', 2),
 ('CDC7', 2),
 ('MTORC1, MTORC2', 2),
 ('G9a and GLP methyltransferases', 2),
 ('Microtubule destabiliser', 2),
 ('MTOR', 2),
 ('PDK1 (PDPK1)', 2),
 ('Amyloid beta20, Amyloid beta40', 2),
 ('Farnesyl-transferase (FNTA)', 2),
 ('MDM2', 2),
 ('SMO', 2),
 ('ATM', 2),
 ('PI3K (class 1), MTORC1, MTORC2', 2),
 ('ALK', 2),
 ('BRD2, BRD3, BRD4, BRDT', 2),
 ('DNA crosslinker', 2),
 ('PI3Kbeta', 2),
 ('SRC, ABL, TEC', 1),
 ('PDGFR, KIT, VEGFR', 1),
 ('MET', 1),
 ('PKC, PPK, FLT1, c-FGR, others', 1),
 ('PB1, SMARCA4, SMARCA2', 1),
 ('JNK2, JNK3', 1),
 ('IGF1R', 1),
 ('BTK', 1),
 ('IKK1, IKK2', 1),
 ('JAK1, JAK2', 1),
 ('JNK1, JNK2, JNK2', 1),
 ('PI3Kgamma', 1),
 ('ERK5,

In [29]:
drug_anno=[]
for idx in gdsc1_info_sel.index:
    name=gdsc1_info_sel.loc[idx,'drug_name']
    pathway=gdsc1_info_sel.loc[idx,'pathway_name']
    target=gdsc1_info_sel.loc[idx,'targets']
    if 'platin' in name:
        drug_anno.append('Platinum')
    elif 'parib' in name:
        drug_anno.append('PARP inhibitor')
    elif 'Microtubule' in target:
        drug_anno.append('Antimicrotubule')
    elif 'Pyrimidine synthesis inhibitor' in target:
        drug_anno.append('Antimetabolite')
    elif 'Alkylating agent' in target:
        drug_anno.append('DNA alkylator')
    elif 'Anthracycline'  in target:
        drug_anno.append('DNA inhibitor')
    elif 'Pyrimidine antimetabolite' in target:
        drug_anno.append('Antimetabolite')
    elif 'Antimetabolite' in target:
        drug_anno.append('Antimetabolite')
    elif 'Anti-metabolite' in target:
        drug_anno.append('Antimetabolite')
    elif 'Dihydrofolate reductase (DHFR)' in target:
        drug_anno.append('Antimetabolite')
    elif 'DNA crosslinker' in target:
        drug_anno.append('DNA alkylator')
    elif 'dsDNA break induction' in target:
        drug_anno.append('DNA inhibitor')
    elif 'DNA replication' in pathway:
        drug_anno.append('DNA inhibitor')
    elif 'DNA alkylating agent' in target:
        drug_anno.append('DNA alkylator')
    elif 'PARP' in target:
        drug_anno.append('PARP inhibitor')
    elif 'TOP' in target:
        drug_anno.append('Topoisomerase inhibitor') 
    elif name in ['Fludarabine','Nelarabine','Dactinomycin','Mitoxantrone',
                 'Teniposide']:
        drug_anno.append('DNA inhibitor')
    elif name in ['Carmustine']:
        drug_anno.append('DNA alkylator')
    elif pathway in ['ERK MAPK signaling','RTK signaling', 'PI3K/MTOR signaling','Cell cycle',
                    'WNT signaling','Protein stability and degradation','WNT signaling',
                     'EGFR signaling','JNK and p38 signaling','p53 pathway','Hormone-related',
                    'Apoptosis regulation','Cytoskeleton','Metabolism','Metabolism',
                     'IGF1R signaling','Genome integrity']:
        drug_anno.append(pathway)
    elif pathway in ['Chromatin histone acetylation','Chromatin histone methylation',
                     'Chromatin other',]:
        drug_anno.append('Chromatin-related')
    elif pathway in ['ABL signaling','Mitosis','IGF1R signaling',]:
        drug_anno.append('Others')   
    else:
        drug_anno.append('Others')
Counter(drug_anno).most_common()

[('Others', 64),
 ('RTK signaling', 26),
 ('PI3K/MTOR signaling', 23),
 ('Chromatin-related', 19),
 ('ERK MAPK signaling', 17),
 ('Cell cycle', 12),
 ('Protein stability and degradation', 8),
 ('DNA inhibitor', 8),
 ('EGFR signaling', 7),
 ('Cytoskeleton', 7),
 ('JNK and p38 signaling', 6),
 ('Apoptosis regulation', 6),
 ('PARP inhibitor', 5),
 ('Antimetabolite', 5),
 ('Antimicrotubule', 5),
 ('Metabolism', 5),
 ('WNT signaling', 4),
 ('Hormone-related', 3),
 ('IGF1R signaling', 3),
 ('Genome integrity', 3),
 ('p53 pathway', 3),
 ('Platinum', 1),
 ('DNA alkylator', 1)]

In [30]:
gdsc1_info_sel['drug_anno']=drug_anno

/var/folders/3g/5zyn6dld5rl1rc3t1fmh0t240000gn/T/ipykernel_54523/3054172337.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gdsc1_info_sel['drug_anno']=drug_anno


In [31]:
drug_anno=[]
for idx in gdsc2_info_sel.index:
    name=gdsc2_info_sel.loc[idx,'drug_name']
    pathway=gdsc2_info_sel.loc[idx,'pathway_name']
    target=gdsc2_info_sel.loc[idx,'targets']
    if 'platin' in name:
        drug_anno.append('Platinum')
    elif 'parib' in name:
        drug_anno.append('PARP inhibitor')
    elif name in ['Irinotecan']:
        drug_anno.append('Topoisomerase inhibitor')
    elif 'Microtubule' in target:
        drug_anno.append('Antimicrotubule')
    elif 'Pyrimidine synthesis inhibitor' in target:
        drug_anno.append('Antimetabolite')
    elif 'Alkylating agent' in target:
        drug_anno.append('DNA alkylator')
    elif 'Anthracycline'  in target:
        drug_anno.append('DNA inhibitor')
    elif 'Pyrimidine antimetabolite' in target:
        drug_anno.append('Antimetabolite')
    elif 'Antimetabolite' in target:
        drug_anno.append('Antimetabolite')
    elif 'Anti-metabolite' in target:
        drug_anno.append('Antimetabolite')
    elif 'Dihydrofolate reductase (DHFR)' in target:
        drug_anno.append('Antimetabolite')
    elif 'DNA crosslinker' in target:
        drug_anno.append('DNA alkylator')
    elif 'dsDNA break induction' in target:
        drug_anno.append('DNA inhibitor')
    elif 'DNA replication' in pathway:
        drug_anno.append('DNA inhibitor')
    elif 'DNA alkylating agent' in target:
        drug_anno.append('DNA alkylator')
    elif 'PARP' in target:
        drug_anno.append('PARP inhibitor')
    elif name in ['Fludarabine','Nelarabine','Dactinomycin','Mitoxantrone',
                 'Teniposide']:
        drug_anno.append('DNA inhibitor')
    elif name in ['Carmustine']:
        drug_anno.append('DNA alkylator')
    elif pathway in ['ERK MAPK signaling','RTK signaling', 'PI3K/MTOR signaling','Cell cycle',
                    'WNT signaling','Protein stability and degradation','WNT signaling',
                     'EGFR signaling','JNK and p38 signaling','p53 pathway','Hormone-related',
                    'Apoptosis regulation','Cytoskeleton','Metabolism','Metabolism',
                     'IGF1R signaling','Genome integrity']:
        drug_anno.append(pathway)
    elif pathway in ['Chromatin histone acetylation','Chromatin histone methylation',
                     'Chromatin other',]:
        drug_anno.append('Chromatin-related')
    elif pathway in ['ABL signaling','Mitosis','IGF1R signaling',]:
        drug_anno.append('Others')   
    else:
        drug_anno.append('Others')
Counter(drug_anno).most_common()

[('Others', 32),
 ('PI3K/MTOR signaling', 21),
 ('Chromatin-related', 18),
 ('DNA inhibitor', 13),
 ('ERK MAPK signaling', 13),
 ('Apoptosis regulation', 12),
 ('RTK signaling', 11),
 ('Cell cycle', 10),
 ('WNT signaling', 9),
 ('Protein stability and degradation', 9),
 ('Genome integrity', 8),
 ('EGFR signaling', 7),
 ('Antimetabolite', 6),
 ('Hormone-related', 5),
 ('Antimicrotubule', 5),
 ('PARP inhibitor', 5),
 ('Metabolism', 4),
 ('IGF1R signaling', 4),
 ('p53 pathway', 4),
 ('Platinum', 3),
 ('DNA alkylator', 2),
 ('JNK and p38 signaling', 2),
 ('Cytoskeleton', 2),
 ('Topoisomerase inhibitor', 1)]

In [32]:
gdsc2_info_sel['drug_anno']=drug_anno

/var/folders/3g/5zyn6dld5rl1rc3t1fmh0t240000gn/T/ipykernel_54523/4142830685.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gdsc2_info_sel['drug_anno']=drug_anno


In [33]:
dict_cid_anno={int(k):v for k,v in zip(gdsc1_info_sel['pubchem'],gdsc1_info_sel['drug_anno'])}
for idx in gdsc2_info_sel.index:
    k=int(gdsc2_info_sel.loc[idx,'pubchem'])
    v=gdsc2_info_sel.loc[idx,'drug_anno']
    dict_cid_anno[k]=v

## CTRP2

In [34]:
drug_anno=[]
for idx in ctrp2_info_sel.index:
    name=ctrp2_info_sel.loc[idx,'cpd_name']
    target= ctrp2_info_sel.loc[idx,'target_or_activity_of_compound']
    gene=ctrp2_info_sel.loc[idx,'gene_symbol_of_protein_target']
    cid=gene=int(ctrp2_info_sel.loc[idx,'cid'])
    if cid in dict_cid_anno:
        drug_anno.append(dict_cid_anno[cid])
    elif 'platin' in name:
        drug_anno.append('Platinum')
    elif 'parib' in name:
        drug_anno.append('PARP inhibitor')
    elif 'topoisomerase' in target:
        drug_anno.append('Topoisomerase inhibitor')
    elif 'tubule' in target :
        drug_anno.append('Antimicrotubule')
    elif 'DNA alkylator' in target:
        drug_anno.append('DNA alkylator')
    elif 'inducer of DNA damage' in target:
        drug_anno.append('DNA inhibitor')
    elif 'DNA crosslinker' in target:
        drug_anno.append('DNA alkylator')
    elif 'inhibitor of DNA synthesis' in target:
        drug_anno.append('Antimetabolite')
    elif 'inhibitor of DNA replication' in target:
        drug_anno.append('DNA inhibitor')
    elif 'inhibitor of thymidylate synthase' in target:
        drug_anno.append('Antimetabolite')
    elif 'inhibitor of cyclin-dependent kinases' in target:
        drug_anno.append('Cell cycle')
    elif 'inhibitor of PI3K and mTOR kinase activity' in target:
        drug_anno.append('PI3K/MTOR signaling')
    elif 'inhibitor of mTOR and PI3K kinase acitivities' in target:
        drug_anno.append('PI3K/MTOR signaling')
    elif 'inhibitor of mTOR'  in target:
        drug_anno.append('PI3K/MTOR signaling')
    elif 'inhibitor of p53-MDM2 interaction' in target:
        drug_anno.append('p53 pathway')
    elif 'inhibitor of HDAC1, HDAC2, HDAC3, HDAC6, and HDAC8' in target:
        drug_anno.append('Chromatin-related')
    elif 'modulator of estrogen receptors'  in target:
        drug_anno.append('Hormone-related')
    else:
        drug_anno.append('Others')
Counter(drug_anno)

Counter({'Others': 91,
         'DNA alkylator': 7,
         'Antimetabolite': 2,
         'Antimicrotubule': 4,
         'Hormone-related': 2,
         'Platinum': 2,
         'DNA inhibitor': 9,
         'Topoisomerase inhibitor': 2,
         'Protein stability and degradation': 2,
         'EGFR signaling': 3,
         'Chromatin-related': 5,
         'PI3K/MTOR signaling': 13,
         'Cell cycle': 7,
         'RTK signaling': 11,
         'ERK MAPK signaling': 3,
         'Apoptosis regulation': 4,
         'p53 pathway': 2,
         'PARP inhibitor': 2,
         'Metabolism': 1,
         'IGF1R signaling': 1})

In [35]:
ctrp2_info_sel['drug_anno']=drug_anno

## CTRP1

In [36]:
drug_anno=[]
for idx in ctrp1_info_sel.index:
    name=ctrp1_info_sel.loc[idx,'cpd_name']
    target= ctrp1_info_sel.loc[idx,'target_or_activity_of_compound']
    gene=ctrp1_info_sel.loc[idx,'gene_symbol_of_protein_target']
    cid=gene=int(ctrp1_info_sel.loc[idx,'pubchem_cid'])
    if cid in dict_cid_anno:
        drug_anno.append(dict_cid_anno[cid])
    elif 'platin' in name:
        drug_anno.append('Platinum')
    elif 'parib' in name:
        drug_anno.append('PARP inhibitor')
    elif 'topoisomerase' in target:
        drug_anno.append('Topoisomerase inhibitor')
    elif 'tubule' in target :
        drug_anno.append('Antimicrotubule')
    elif 'tubulin' in target :
        drug_anno.append('Antimicrotubule')
    elif 'DNA alkylator' in target:
        drug_anno.append('DNA alkylator')
    elif 'inducer of DNA damage' in target:
        drug_anno.append('DNA inhibitor')
    elif 'DNA crosslinker' in target:
        drug_anno.append('DNA alkylator')
    elif 'inhibitor of DNA synthesis' in target:
        drug_anno.append('Antimetabolite')
    elif 'thymidylate' in target:
        drug_anno.append('Antimetabolite')
    elif 'inhibitor of DNA replication' in target:
        drug_anno.append('DNA inhibitor')
    elif 'inhibitor of thymidylate synthase' in target:
        drug_anno.append('Antimetabolite')
    elif 'inhibitor of cyclin-dependent kinases' in target:
        drug_anno.append('Cell cycle')
    elif 'inhibitor of PI3K and mTOR kinase activity' in target:
        drug_anno.append('PI3K/MTOR signaling')
    elif 'inhibitor of mTOR and PI3K kinase acitivities' in target:
        drug_anno.append('PI3K/MTOR signaling')
    elif 'inhibitor of mTOR'  in target:
        drug_anno.append('PI3K/MTOR signaling')
    elif 'inhibitor of p53-MDM2 interaction' in target:
        drug_anno.append('p53 pathway')
    elif 'inhibitor of HDAC1, HDAC2, HDAC3, HDAC6, and HDAC8' in target:
        drug_anno.append('Chromatin-related')
    elif 'modulator of estrogen receptors'  in target:
        drug_anno.append('Hormone-related')
    elif 'antagonist of androgen receptor'  in target:
        drug_anno.append('Hormone-related')
    else:
        drug_anno.append('Others')
Counter(drug_anno)

Counter({'Others': 61,
         'Apoptosis regulation': 2,
         'Antimicrotubule': 2,
         'PI3K/MTOR signaling': 2,
         'Cell cycle': 1,
         'Hormone-related': 4,
         'RTK signaling': 2,
         'Protein stability and degradation': 3,
         'DNA inhibitor': 2,
         'Metabolism': 1,
         'Antimetabolite': 2,
         'Topoisomerase inhibitor': 1,
         'IGF1R signaling': 1,
         'DNA alkylator': 1,
         'PARP inhibitor': 3,
         'ERK MAPK signaling': 1,
         'Chromatin-related': 1})

In [37]:
ctrp1_info_sel['drug_anno']=drug_anno

## PRISM

In [38]:
prism_info_sel['moa']=prism_info_sel['moa'].fillna('NoData')

In [39]:
drug_anno=[]
for idx in prism_info_sel.index:
    name=prism_info_sel.loc[idx,'PRISM.drugid']
    cid=int(prism_info_sel.loc[idx,'cid'])
    moa=prism_info_sel.loc[idx,'moa']
    if cid in dict_cid_anno:
        drug_anno.append(dict_cid_anno[cid])
    elif 'platin' in name:
        drug_anno.append('Platinum')
    elif 'parib' in name:
        drug_anno.append('PARP inhibitor')
    elif 'topoisomerase inhibitor' in moa:
        drug_anno.append('Topoisomerase inhibitor')
    elif 'tubul' in moa :
        drug_anno.append('Antimicrotubule')
    elif 'DNA alkylating agent' in moa:
        drug_anno.append('DNA alkylator')
    elif 'DNA synthesis inhibitor' in moa:
        drug_anno.append('Antimetabolite')
    elif 'ribonucleotide reductase inhibitor' in moa:
        drug_anno.append('Antimetabolite')
    elif 'thymidylate synthase inhibitor' in moa:
        drug_anno.append('Antimetabolite')
    elif 'adenosine deaminase inhibitor' in moa:
        drug_anno.append('Antimetabolite')
    elif 'dihydrofolate reductase inhibitor	DHFR' in moa:
        drug_anno.append('Antimetabolite')
    elif 'dihydrofolate reductase inhibitor' in moa:
        drug_anno.append('Antimetabolite')
    elif name in ['azacitidine']:
        drug_anno.append('Antimetabolite')
    elif 'PARP' in moa:
        drug_anno.append('PARP inhibitor')
    elif 'thymidylate synthase inhibitor' in moa:
        drug_anno.append('Antimetabolite')
    elif  'DNA intercalating agent' in moa:
        drug_anno.append('other DNA inhibitor')
    elif 'purine antagonist' in moa:
        drug_anno.append('Antimetabolite')
    elif 'dihydropyrimidine dehydrogenase inhibitor' in moa:
        drug_anno.append('Antimetabolite')
    elif 'DNA damage inducer' in moa:
        drug_anno.append('DNA inhibitor')
    elif 'DNA inhibitor' in moa:
        drug_anno.append('DNA inhibitor')
    elif moa in ['aromatase inhibitor','androgen receptor antagonist',
                'estrogen receptor antagonist, selective estrogen receptor modulator (SERM)',
                'androgen biosynthesis inhibitor']:
        drug_anno.append('Hormone-related')
    elif moa in ['EGFR inhibitor']:
        drug_anno.append('EGFR signaling')
    elif moa in ['proteasome inhibitor']:
        drug_anno.append('Protein stability and degradation')    
    elif moa in ['HDAC inhibitor']:
        drug_anno.append('Chromatin-related')    
    elif moa in ['CDK inhibitor']:
        drug_anno.append('Cell cycle') 
    elif moa in ['MEK inhibitor']:
        drug_anno.append('ERK MAPK signaling') 
    elif moa in ['mTOR inhibitor']:
        drug_anno.append('PI3K/MTOR signaling') 
    else:
        drug_anno.append('Others')
Counter(drug_anno).most_common()

[('Others', 43),
 ('Antimetabolite', 23),
 ('Hormone-related', 15),
 ('DNA inhibitor', 12),
 ('Antimicrotubule', 12),
 ('Topoisomerase inhibitor', 10),
 ('EGFR signaling', 10),
 ('DNA alkylator', 8),
 ('RTK signaling', 6),
 ('Protein stability and degradation', 6),
 ('Chromatin-related', 5),
 ('ERK MAPK signaling', 5),
 ('Cell cycle', 4),
 ('PI3K/MTOR signaling', 4),
 ('PARP inhibitor', 4),
 ('Platinum', 3),
 ('Apoptosis regulation', 2)]

In [40]:
prism_info_sel['drug_anno']=drug_anno

## gCSI

In [41]:
drug_anno=[]
for idx in gcsi_info_sel.index:
    name=gcsi_info_sel.loc[idx,'drugid']
    cid=int(gcsi_info_sel.loc[idx,'cid'])
    if cid in dict_cid_anno:
        drug_anno.append(dict_cid_anno[cid])
    elif 'platin' in name:
        drug_anno.append('Platinum')
    elif name in ['Vincristine','Vincaleukoblastine']:
        drug_anno.append('Antimicrotubule')
    elif name in ['5-Fluorouracil','Gemcitabine']:
        drug_anno.append('Antimetabolite')
    elif name in ['Erlotinib']:
        drug_anno.append('EGFR signaling')
    elif name in ['Crizotinib']:
        drug_anno.append('RTK signaling')
    else:
        drug_anno.append('Others')
Counter(drug_anno)

Counter({'Antimetabolite': 3,
         'ERK MAPK signaling': 3,
         'Others': 14,
         'Cell cycle': 2,
         'PI3K/MTOR signaling': 6,
         'Protein stability and degradation': 3,
         'WNT signaling': 1,
         'Platinum': 1,
         'RTK signaling': 1,
         'Antimicrotubule': 3,
         'EGFR signaling': 3,
         'DNA inhibitor': 1,
         'Genome integrity': 1,
         'Chromatin-related': 1})

In [42]:
gcsi_info_sel['drug_anno']=drug_anno

/var/folders/3g/5zyn6dld5rl1rc3t1fmh0t240000gn/T/ipykernel_54523/3916020972.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gcsi_info_sel['drug_anno']=drug_anno


## Write out

In [43]:
gdsc1_info_sel['drug_name_common']=['GDSC1_'+x+'_'+str(y) for x,y in zip(gdsc1_info_sel['drug_name'],gdsc1_info_sel['drug_id']) ]
gdsc2_info_sel['drug_name_common']=['GDSC2_'+x+'_'+str(y) for x,y in zip(gdsc2_info_sel['drug_name'],gdsc2_info_sel['drug_id']) ]
ctrp1_info_sel['drug_name_common']=['CTRP1_'+x+'_'+str(y) for x,y in zip(ctrp1_info_sel['cpd_name'],ctrp1_info_sel['master_cpd_id']) ]
ctrp2_info_sel['drug_name_common']=['CTRP2_'+x+'_'+str(y) for x,y in zip(ctrp2_info_sel['cpd_name'],ctrp2_info_sel['master_cpd_id']) ]
prism_info_sel['drug_name_common']=['PRISM_'+x+'_'+str(y) for x,y in zip(prism_info_sel['PRISM.drugid'],prism_info_sel['broad_id']) ]
gcsi_info_sel['drug_name_common']=['gCSI_'+x+'_CID'+str(y) for x,y in zip(gcsi_info_sel.index,gcsi_info_sel['cid']) ]

/var/folders/3g/5zyn6dld5rl1rc3t1fmh0t240000gn/T/ipykernel_54523/2996626426.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gdsc1_info_sel['drug_name_common']=['GDSC1_'+x+'_'+str(y) for x,y in zip(gdsc1_info_sel['drug_name'],gdsc1_info_sel['drug_id']) ]
/var/folders/3g/5zyn6dld5rl1rc3t1fmh0t240000gn/T/ipykernel_54523/2996626426.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gdsc2_info_sel['drug_name_common']=['GDSC2_'+x+'_'+str(y) for x,y in zip(gdsc2_info_sel['drug_name'],gdsc2_info_sel['drug_id']

In [44]:
gdsc1_info_sel.to_csv('../data/GDSC1_drug_category.txt',sep='\t')
gdsc2_info_sel.to_csv('../data/GDSC2_drug_category.txt',sep='\t')
ctrp1_info_sel.to_csv('../data/CTRP1_drug_category.txt',sep='\t')
ctrp2_info_sel.to_csv('../data/CTRP2_drug_category.txt',sep='\t')
prism_info_sel.to_csv('../data/PRISM_drug_category.txt',sep='\t')
gcsi_info_sel.to_csv('../data/gCSI_drug_category.txt',sep='\t')

In [45]:
# Relation to the primary information
gdsc1_info_sel['drug_anno_primary']=gdsc1_info_sel['pathway_name']
gdsc2_info_sel['drug_anno_primary']=gdsc2_info_sel['pathway_name']
ctrp2_info_sel['drug_anno_primary']=ctrp2_info_sel['target_or_activity_of_compound']
ctrp1_info_sel['drug_anno_primary']=ctrp1_info_sel['target_or_activity_of_compound']
prism_info_sel['drug_anno_primary']=prism_info_sel['moa']
gcsi_info_sel['drug_anno_primary']="None"

/var/folders/3g/5zyn6dld5rl1rc3t1fmh0t240000gn/T/ipykernel_54523/589786097.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gdsc1_info_sel['drug_anno_primary']=gdsc1_info_sel['pathway_name']
/var/folders/3g/5zyn6dld5rl1rc3t1fmh0t240000gn/T/ipykernel_54523/589786097.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gdsc2_info_sel['drug_anno_primary']=gdsc2_info_sel['pathway_name']
/var/folders/3g/5zyn6dld5rl1rc3t1fmh0t240000gn/T/ipykernel_54523/589786097.py:7: SettingWithCopyWarning: 
A value is trying 

In [46]:
add1=gdsc1_info_sel[['drug_name_common','drug_anno','drug_anno_primary','pubchem']].rename(columns={'pubchem':'cid'})
add2=gdsc2_info_sel[['drug_name_common','drug_anno','drug_anno_primary','pubchem']].rename(columns={'pubchem':'cid'})
add3=ctrp1_info_sel[['drug_name_common','drug_anno','drug_anno_primary','pubchem_cid']].rename(columns={'pubchem_cid':'cid'})
add4=ctrp2_info_sel[['drug_name_common','drug_anno','drug_anno_primary','cid']]
add5=prism_info_sel[['drug_name_common','drug_anno','drug_anno_primary','cid']]
add6=gcsi_info_sel[['drug_name_common','drug_anno','drug_anno_primary','cid']]

In [47]:
dfw=pd.concat([add1,add2,add3,add4,add5,add6])

In [48]:
Counter(dfw['drug_anno']).most_common()

[('Others', 305),
 ('PI3K/MTOR signaling', 69),
 ('RTK signaling', 57),
 ('Chromatin-related', 49),
 ('DNA inhibitor', 45),
 ('ERK MAPK signaling', 42),
 ('Antimetabolite', 41),
 ('Cell cycle', 36),
 ('Protein stability and degradation', 31),
 ('Antimicrotubule', 31),
 ('EGFR signaling', 30),
 ('Hormone-related', 29),
 ('Apoptosis regulation', 26),
 ('PARP inhibitor', 19),
 ('DNA alkylator', 19),
 ('WNT signaling', 14),
 ('Topoisomerase inhibitor', 14),
 ('Genome integrity', 12),
 ('Metabolism', 11),
 ('Platinum', 10),
 ('IGF1R signaling', 9),
 ('Cytoskeleton', 9),
 ('p53 pathway', 9),
 ('JNK and p38 signaling', 8)]

In [49]:
dfw.to_csv('../data/Drug_annotaion_integrated.txt',sep='\t',index=False)

# Drug AUC integrated

In [50]:
gdsc1_info_sel=pd.read_csv('../data/GDSC1_drug_category.txt',sep='\t')
gdsc2_info_sel=pd.read_csv('../data/GDSC2_drug_category.txt',sep='\t')
ctrp1_info_sel=pd.read_csv('../data/CTRP1_drug_category.txt',sep='\t')
ctrp2_info_sel=pd.read_csv('../data/CTRP2_drug_category.txt',sep='\t')
prism_info_sel=pd.read_csv('../data/PRISM_drug_category.txt',sep='\t')
gcsi_info_sel=pd.read_csv('../data/gCSI_drug_category.txt',sep='\t')

In [51]:
gdsc1=pd.read_excel('../download/GDSC1_fitted_dose_response_25Feb20.xlsx')

In [52]:
gdsc1_sel=gdsc1[gdsc1['DRUG_ID'].isin(gdsc1_info_sel['drug_id'])]

In [53]:
gdsc1_sel['Drug_name_id']=['GDSC1_'+x+'_'+str(y) for x,y in zip(gdsc1_sel['DRUG_NAME'],gdsc1_sel['DRUG_ID'])]

/var/folders/3g/5zyn6dld5rl1rc3t1fmh0t240000gn/T/ipykernel_54523/2643892336.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gdsc1_sel['Drug_name_id']=['GDSC1_'+x+'_'+str(y) for x,y in zip(gdsc1_sel['DRUG_NAME'],gdsc1_sel['DRUG_ID'])]


In [54]:
dict_cell_drug_val={}
val='AUC'
for idx in gdsc1_sel.index:
    cell=gdsc1_sel.loc[idx,'COSMIC_ID']
    drug=gdsc1_sel.loc[idx,'Drug_name_id']
    if cell not in dict_cell_drug_val:
        dict_cell_drug_val[cell]={}
    else:
        dict_cell_drug_val[cell][drug]=gdsc1_sel.loc[idx,val]

In [55]:
gdsc1_add=pd.DataFrame(dict_cell_drug_val).sort_index().sort_index(axis=1).T
gdsc1_add

,GDSC1_(5Z)-7-Oxozeaenol_1242,GDSC1_5-Fluorouracil_179,GDSC1_A-443654_86,GDSC1_A-770041_55,GDSC1_AICA Ribonucleotide_1001,GDSC1_AKT inhibitor VIII_171,GDSC1_AKT inhibitor VIII_228,GDSC1_AR-42_272,GDSC1_AS601245_207,GDSC1_AS605240_224,...,GDSC1_XAV939_1268,GDSC1_XMD8-85_106,GDSC1_XMD8-92_1164,GDSC1_Y-39983_309,GDSC1_YK-4-279_1239,GDSC1_YM201636_310,GDSC1_Z-LLNle-CHO_45,GDSC1_ZM447439_1050,GDSC1_ZSTK474_223,GDSC1_Zibotentan_266
683665,0.566264,0.408888,0.913908,0.795985,0.568754,0.938230,0.505289,0.333898,0.830618,0.554403,...,0.972283,0.787667,0.841502,0.849709,NaN,0.795951,0.749002,0.828956,0.303362,0.987224
683667,NaN,0.897167,NaN,NaN,0.948461,0.984378,0.894289,0.659044,0.941653,0.896050,...,0.984642,NaN,0.806652,0.897095,NaN,0.956405,NaN,0.924510,0.815931,0.985981
684052,0.641645,0.963662,NaN,NaN,0.888970,0.974401,0.942074,0.687056,0.889795,0.853207,...,0.978493,NaN,0.954137,0.980571,0.649946,0.948255,NaN,0.978731,0.828291,0.991637
684055,0.791229,0.847243,0.911974,0.966585,0.795625,0.950905,0.735332,0.452680,0.870549,0.931199,...,0.983505,0.984884,0.884187,0.916355,0.734536,0.742204,0.967817,0.704326,0.709809,0.983258
684057,0.748476,0.677593,0.972222,0.983873,0.895263,0.917635,0.497848,0.223127,0.877500,0.729284,...,0.973736,0.924651,0.867384,0.879714,0.659554,0.649945,0.974184,0.817735,0.385414,0.970608
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1660034,0.507933,0.589616,NaN,NaN,0.933854,NaN,0.761827,0.571169,NaN,0.667013,...,0.856987,NaN,0.869998,0.864742,0.698488,0.773071,NaN,0.865617,0.712223,0.972329
1660035,0.978622,0.659344,NaN,NaN,0.918059,NaN,0.875047,0.355107,NaN,0.711258,...,0.965019,NaN,NaN,0.975891,0.974920,0.910276,NaN,0.942055,0.872693,0.967279
1660036,0.738007,0.617234,NaN,NaN,0.884238,NaN,0.730505,0.976338,NaN,0.608856,...,0.979175,NaN,0.951069,0.968024,0.861793,0.955228,NaN,0.814820,0.432177,0.981412
1674021,0.632869,0.865460,NaN,NaN,0.865364,NaN,0.928372,0.656471,NaN,0.822525,...,0.941508,NaN,0.909621,0.961090,0.694510,0.946183,NaN,0.955444,0.863962,0.983048


## GDSC2

In [56]:
gdsc2=pd.read_csv('../download/GDSC2_fitted_dose_response_25Feb20.csv')

In [57]:
gdsc2_sel=gdsc2[gdsc2['DRUG_ID'].isin(gdsc2_info_sel['drug_id'])]
gdsc2_sel['Drug_name_id']=['GDSC2_'+x+'_'+str(y) for x,y in zip(gdsc2_sel['DRUG_NAME'],gdsc2_sel['DRUG_ID'])]

/var/folders/3g/5zyn6dld5rl1rc3t1fmh0t240000gn/T/ipykernel_54523/1860072588.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gdsc2_sel['Drug_name_id']=['GDSC2_'+x+'_'+str(y) for x,y in zip(gdsc2_sel['DRUG_NAME'],gdsc2_sel['DRUG_ID'])]


In [58]:
dict_cell_drug_val={}
val='AUC'
for idx in gdsc2_sel.index:
    cell=gdsc2_sel.loc[idx,'COSMIC_ID']
    drug=gdsc2_sel.loc[idx,'Drug_name_id']
    if cell not in dict_cell_drug_val:
        dict_cell_drug_val[cell]={}
    else:
        dict_cell_drug_val[cell][drug]=gdsc2_sel.loc[idx,val]

In [59]:
gdsc2_add=pd.DataFrame(dict_cell_drug_val).sort_index().sort_index(axis=1).T
gdsc2_add

,GDSC2_5-Fluorouracil_1073,GDSC2_ABT737_1910,GDSC2_AGI-5198_1913,GDSC2_AGI-6780_1634,GDSC2_AMG-319_2045,GDSC2_AZ6102_2109,GDSC2_AZ960_1250,GDSC2_AZD3759_1915,GDSC2_AZD4547_1786,GDSC2_AZD5363_1916,...,GDSC2_Vinorelbine_2048,GDSC2_Vorinostat_1012,GDSC2_WEHI-539_1997,GDSC2_WIKI4_1940,GDSC2_WZ4003_1614,GDSC2_Wee1 Inhibitor_1046,GDSC2_Wnt-C59_1622,GDSC2_XAV939_1268,GDSC2_YK-4-279_1239,GDSC2_ZM447439_1050
683665,0.988746,0.491973,0.991472,0.921248,0.990548,0.855110,0.670322,0.990219,0.968525,0.961817,...,0.645842,0.744778,0.306364,0.841007,0.945011,0.713712,0.852600,0.939151,0.695062,NaN
683667,0.976741,0.655735,0.972966,0.972888,0.971282,0.837857,0.860038,0.989522,0.994262,0.876508,...,0.710986,0.695245,0.922247,0.948386,0.882791,0.827897,0.961412,0.942191,0.784064,NaN
684052,0.949240,0.646030,0.972733,0.958843,0.988567,0.794876,0.807361,0.984985,0.948738,0.935697,...,0.640397,0.800694,0.949760,0.949546,0.955502,0.837360,0.969871,0.973265,0.617951,NaN
684057,0.959395,NaN,NaN,0.948520,NaN,NaN,0.730258,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.825403,NaN,0.965422,0.973363,NaN,NaN
684059,0.858744,0.785829,0.966310,0.963419,0.955631,0.848514,0.756592,0.986403,0.982099,0.934719,...,0.529711,0.785805,0.948819,0.944875,0.915169,0.881386,0.961724,0.985809,0.670564,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1660034,0.832363,0.918031,0.955663,0.976561,NaN,0.925704,0.951805,0.947328,0.966178,0.882164,...,0.760526,0.882738,0.955368,0.881543,0.975864,0.918233,0.954496,0.949847,0.757741,NaN
1660035,0.963426,0.938137,0.949747,0.941915,NaN,NaN,0.863267,0.969401,0.981937,0.947255,...,NaN,0.784177,0.977847,0.934910,0.963458,0.971475,0.955843,0.968306,0.875885,NaN
1660036,0.924514,NaN,NaN,0.983145,NaN,NaN,0.957422,NaN,0.971722,NaN,...,0.884967,NaN,NaN,NaN,0.987716,NaN,0.971806,0.993065,NaN,NaN
1674021,0.831864,0.914564,0.971497,0.971378,0.968631,0.848079,0.838834,0.970680,0.961853,0.829375,...,0.627238,0.747011,0.949813,0.905999,0.943055,0.758515,0.979652,0.959474,0.760292,NaN


## CTRP2

In [60]:
ctrp2_exp=pd.read_csv('../download/v20.meta.per_experiment.txt',sep='\t')
dict_expid_cclid={}
for idx in ctrp2_exp.index:
    dict_expid_cclid[ctrp2_exp.loc[idx,'experiment_id']]=ctrp2_exp.loc[idx,'master_ccl_id']

In [61]:
ctrp2_val=pd.read_csv('../download/v20.data.curves_post_qc.txt',sep='\t')
ctrp2_val['master_ccl_id']=[dict_expid_cclid[x] for x in ctrp2_val['experiment_id']]

In [62]:
dict_cid_cpd_val={}
for idx in ctrp2_val.index:
    val=ctrp2_val.loc[idx,'area_under_curve']
    cid=ctrp2_val.loc[idx,'master_ccl_id']
    cpd=ctrp2_val.loc[idx,'master_cpd_id']
    if cid not in dict_cid_cpd_val:
        dict_cid_cpd_val[cid]={}
    else:
        dict_cid_cpd_val[cid][cpd]=val
        
ctrp2_auc=pd.DataFrame(dict_cid_cpd_val)

In [63]:
ctrp2_cell=pd.read_csv('../download/v20.meta.per_cell_line.txt',sep='\t')
ctrp2_cell['master_ccl_id'].duplicated().sum()
dict_cellid_cname_ctrp2={k:v for k,v in zip(ctrp2_cell['master_ccl_id'],ctrp2_cell['ccl_name'])}

In [64]:
ctrp2_info_sel
dict_cpdid_name_ctrp2={k:v for k,v in zip(ctrp2_info_sel['master_cpd_id'],ctrp2_info_sel['cpd_name'])}

In [65]:
ctrp2_auc_sel=ctrp2_auc.loc[ctrp2_auc.index.isin(dict_cpdid_name_ctrp2),:]
ctrp2_auc_sel=ctrp2_auc_sel.rename(columns=dict_cellid_cname_ctrp2).sort_index(axis=1)

In [66]:
dids=[]
for idx in ctrp2_auc_sel.index:
    did='CTRP2_'+dict_cpdid_name_ctrp2[idx]+'_'+str(idx)
    dids.append(did)
ctrp2_auc_sel.index=dids

In [67]:
ctrp2_auc_sel=ctrp2_auc_sel.sort_index()

## CTRP1

In [68]:
ctrp1_val=pd.read_csv('../download/v10.D3.area_under_conc_curve.txt',sep='\t')

In [69]:
dict_cid_cpd_val={}
for idx in ctrp1_val.index:
    val=ctrp1_val.loc[idx,'area_under_curve']
    cid=ctrp1_val.loc[idx,'ccl_name']
    cpd=ctrp1_val.loc[idx,'cpd_name']
    if cid not in dict_cid_cpd_val:
        dict_cid_cpd_val[cid]={}
    else:
        dict_cid_cpd_val[cid][cpd]=val        
ctrp1_auc=pd.DataFrame(dict_cid_cpd_val)

In [70]:
dict_cpdname_did_ctrp1={v:k for k,v in zip(ctrp1_info_sel['master_cpd_id'],ctrp1_info_sel['cpd_name'])}

In [71]:
ctrp1_auc_sel=ctrp1_auc[ctrp1_auc.index.isin(ctrp1_info_sel['cpd_name'])]
ctrp1_auc_sel=ctrp1_auc_sel.sort_index().sort_index(axis=1)

In [72]:
dids=[]
for idx in ctrp1_auc_sel.index:
    did='CTRP1_'+str(idx)+'_'+str(dict_cpdname_did_ctrp1[idx])
    dids.append(did)
ctrp1_auc_sel.index=dids

In [73]:
ctrp1_auc_sel

,5637,A2780,A375,A549,AGS,AMO1,BEN,BL70,C2BBE1,CAL12T,...,SW948,T3M10,THP1,TYKNU,U2OS,U937,UMUC3,VMRCLCD,VMRCRCW,WSUDLCL2
CTRP1_2-deoxyglucose_347781,6.2317,NaN,NaN,6.5936,NaN,NaN,7.2503,6.10170,6.1555,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,6.9127,6.55890,6.7850,NaN
CTRP1_ABT-737_411738,5.2349,6.4910,7.1257,6.2269,3.7019,7.0969,3.5973,5.47980,5.2244,6.7488,...,5.2419,1.4003,5.77030,6.3093,6.7599,4.9088,5.3930,3.77400,4.8952,3.18870
CTRP1_ABT-751_417983,1.3811,2.2385,2.6517,3.7262,2.0065,1.1897,5.8026,0.15431,NaN,3.9351,...,4.2293,4.6704,2.53670,3.2594,4.4100,1.1265,3.7863,0.80486,4.8160,0.56658
CTRP1_ATRA_411870,NaN,6.6698,5.9579,NaN,7.5154,9.0956,NaN,NaN,NaN,6.7019,...,5.4495,6.3662,3.89240,6.2165,6.7517,3.9180,NaN,NaN,NaN,4.33330
CTRP1_BEZ235_348992,NaN,3.4951,3.8438,NaN,3.3303,4.9585,NaN,NaN,NaN,3.3834,...,2.4341,5.0682,2.89110,3.0690,3.7451,2.9151,NaN,NaN,NaN,2.14060
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
CTRP1_valdecoxib_32372,NaN,4.9585,6.6086,NaN,6.8206,7.3084,NaN,NaN,NaN,5.0076,...,5.4862,5.1755,4.50830,5.3782,NaN,3.8451,NaN,NaN,NaN,5.47510
CTRP1_veliparib_606034,NaN,NaN,6.3404,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,6.6674,5.66240,6.3382,NaN,7.4383,NaN,NaN,NaN,6.85490
CTRP1_vismodegib_397704,6.6173,7.2539,6.9366,7.0051,6.2829,8.5852,6.8776,7.04650,6.6561,6.9891,...,6.8072,6.6719,6.07770,6.2801,6.8144,6.5591,6.8400,6.75210,6.6989,6.79200
CTRP1_vorinostat_56554,2.4331,1.9301,2.7053,2.7953,1.3732,2.0972,2.5882,0.97163,2.6140,3.4094,...,1.1170,3.7680,0.98992,2.7057,2.0930,1.0660,4.6142,1.12710,3.6560,0.93040


## PRISM

In [74]:
prism=pd.read_csv('../download/secondary-screen-dose-response-curve-parameters.csv')

/Users/tshiro/.pyenv/versions/miniforge3-4.10/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (14,15) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [75]:
dict_bid_did_val={}
for idx in prism.index:
    bid=prism.loc[idx,'broad_id']
    did=prism.loc[idx,'depmap_id']
    val=prism.loc[idx,'auc']
    if bid not in dict_bid_did_val:
        dict_bid_did_val[bid]={}
    else:
        dict_bid_did_val[bid][did]=val

In [76]:
prism_auc=pd.DataFrame(dict_bid_did_val).T
prism_auc

,ACH-000320,ACH-001145,ACH-000873,ACH-000855,ACH-000913,ACH-000558,ACH-000593,ACH-000488,ACH-000832,ACH-000781,...,ACH-000924,ACH-000628,ACH-000510,ACH-000086,ACH-000947,ACH-000917,ACH-000454,ACH-000030,ACH-000476,ACH-000826
BRD-K71847383-001-12-5,1.240300,1.472333,1.207160,1.229332,1.130378,1.436740,1.328064,1.200396,1.335735,1.467798,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
BRD-A98990573-003-01-3,1.565786,1.525007,1.333858,1.461459,1.455895,1.581816,1.544465,1.249404,1.462856,1.562774,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
BRD-K47832606-001-30-1,0.336463,0.485618,0.548483,0.631609,0.647397,0.580288,0.645974,0.598061,0.564635,0.690942,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
BRD-K72951360-001-01-4,0.651509,0.505583,0.591650,0.617227,0.809460,0.826771,0.752404,0.630702,0.647558,0.707443,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
BRD-K33127281-001-05-6,1.379432,0.968500,1.194919,1.209085,1.165317,1.260609,1.135452,1.121895,1.172720,1.236926,...,1.086418,1.255210,1.131118,0.977359,0.987091,1.40125,0.924889,1.076209,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
BRD-K85402309-043-01-9,0.866265,0.862640,0.781424,1.000000,0.694885,0.943005,0.948937,0.796833,0.946861,NaN,...,1.000000,1.000000,0.988552,0.718694,0.758549,NaN,0.833390,0.780134,0.929951,0.891253
BRD-K88510285-001-17-8,0.492455,0.633637,0.457427,0.534181,0.720609,0.511024,0.687134,0.771501,0.515982,1.000000,...,0.839772,0.717733,0.729825,0.617289,0.538059,NaN,0.509546,0.591776,0.758900,1.000000
BRD-K92723993-001-17-4,1.000000,1.000000,0.989859,NaN,1.000000,NaN,1.000000,1.000000,0.972794,1.000000,...,1.000000,1.000000,1.000000,1.000000,NaN,NaN,0.936010,0.911568,0.915373,1.000000
BRD-M97302542-001-04-4,1.000000,0.996181,1.000000,1.000000,0.926474,0.847692,0.961398,0.992998,0.987591,0.864479,...,0.913766,1.000000,0.881420,1.000000,NaN,NaN,1.000000,0.975725,1.000000,1.000000


In [77]:
dict_rename={bid:'PRISM_'+did+'_'+bid for bid,did in zip(prism_info_sel['broad_id'],prism_info_sel['PRISM.drugid'])}

In [78]:
prism_auc=prism_auc[prism_auc.index.isin(dict_rename)]
prism_auc=prism_auc.rename(index=dict_rename).sort_index()

In [79]:
prism_auc

,ACH-000320,ACH-001145,ACH-000873,ACH-000855,ACH-000913,ACH-000558,ACH-000593,ACH-000488,ACH-000832,ACH-000781,...,ACH-000924,ACH-000628,ACH-000510,ACH-000086,ACH-000947,ACH-000917,ACH-000454,ACH-000030,ACH-000476,ACH-000826
PRISM_5-fluorouracil_BRD-K24844714-001-24-5,NaN,0.795586,0.835850,0.840685,1.000000,0.887022,0.883818,0.861537,0.836807,NaN,...,NaN,0.883582,0.885603,0.913952,0.813660,1.278939,NaN,NaN,1.135872,NaN
PRISM_SN-38_BRD-K89561498-001-01-7,NaN,0.023993,0.072522,0.210801,NaN,0.585773,0.521405,NaN,NaN,0.527425,...,0.416981,0.422974,0.228873,0.432238,NaN,0.257515,0.267526,0.136627,0.262572,0.730666
PRISM_abemaciclib_BRD-K33622447-066-01-9,0.708827,0.792650,1.000000,0.868010,1.000000,0.903414,0.772102,0.777535,0.868960,0.958988,...,1.000000,1.000000,0.892494,1.000000,1.000000,0.874214,0.806905,0.981584,0.762830,0.809008
PRISM_abiraterone-acetate_BRD-K24048528-001-02-5,NaN,1.501057,1.109186,1.115573,NaN,1.129441,NaN,1.526493,1.135149,1.494643,...,NaN,NaN,NaN,NaN,NaN,NaN,1.137409,NaN,NaN,NaN
PRISM_abiraterone_BRD-K50071428-001-03-3,NaN,1.627181,1.328991,1.673773,0.968556,NaN,1.522590,1.534069,1.615634,1.512218,...,1.250617,1.262884,1.244644,1.470435,1.373420,NaN,1.395685,NaN,0.932627,1.471053
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
PRISM_vinflunine_BRD-K64120610-001-01-4,0.564117,0.470755,0.680998,0.581386,0.571733,0.635223,0.661573,0.475758,0.550671,0.823145,...,0.659767,0.635454,0.610683,0.676567,0.595393,0.561611,0.635369,0.588355,0.607589,0.705564
PRISM_vinorelbine_BRD-K69280563-001-01-8,NaN,0.615808,0.650861,0.600577,0.707673,0.631822,0.761449,0.579800,0.594327,0.757059,...,0.607389,NaN,0.694411,0.614668,NaN,0.673345,NaN,0.543739,0.777762,0.554019
PRISM_vismodegib_BRD-K44827188-001-06-0,0.990467,1.000000,0.907179,0.935375,0.916082,0.935450,0.902910,1.000000,0.960454,1.308591,...,0.973182,0.951078,0.931225,0.979384,1.000000,1.570836,0.914508,0.815378,0.907342,0.949618
PRISM_vorinostat_BRD-K81418486-001-44-2,0.793436,NaN,0.841797,0.923458,0.838687,0.888587,NaN,0.932778,0.815432,0.957652,...,0.945146,0.873624,0.970837,1.000000,0.746280,NaN,0.813133,0.706647,0.865833,1.000000


## gCSI

In [80]:
gcsi=pd.read_csv('../download/gCSI2019_pharmacoDB_profiles.tsv',sep='\t')
gcsi['Norm_CellLineName']=[x.split('_')[0] for x in gcsi['Unnamed: 0']]
gcsi['DrugName']=[x.split('_')[1] for x in gcsi['Unnamed: 0']]

In [81]:
dict_cell_drug_val={}
val='GR_AOC_published'
for idx in gcsi.index:
    cell=gcsi.loc[idx,'Norm_CellLineName']
    drug=gcsi.loc[idx,'DrugName']
    if cell not in dict_cell_drug_val:
        dict_cell_drug_val[cell]={}
    else:
        dict_cell_drug_val[cell][drug]=gcsi.loc[idx,val]

In [82]:
gcsi_add=pd.DataFrame(dict_cell_drug_val).sort_index().sort_index(axis=1).T
gcsi_add.index=[x.replace('-','').replace(' ','').replace(';','').upper() for x in gcsi_add.index]
gcsi_add

,17-AAG,5-FU,AZ-628,AZD-7762,AZD-8055,Azacitidine,Bortezomib,Bosutinib,CAL-101,CHIR-99021,...,OligomycinA,PF-4708671,PLX-4720,Paclitaxel,Palbociclib,TGX-221,Tivantinib,Vinblastine,Vincristine,Vorinostat
22RV1,NaN,0.279667,0.166333,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.014222,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
769P,NaN,NaN,NaN,NaN,NaN,0.171000,NaN,NaN,0.114333,NaN,...,0.246111,NaN,NaN,0.437667,0.064444,0.156000,0.413333,0.152444,-0.034000,0.140556
8305C,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.149889,NaN,...,0.574000,NaN,NaN,0.652556,NaN,NaN,0.732333,NaN,0.159778,0.465000
8505C,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.047778,NaN,...,0.251444,NaN,NaN,0.558333,NaN,NaN,0.391000,NaN,0.143000,0.340333
A253,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
WSUDLCL2,NaN,NaN,NaN,NaN,NaN,0.345889,NaN,NaN,0.222667,NaN,...,0.124222,NaN,NaN,1.016444,0.132333,0.062889,0.674444,0.465000,0.568667,0.777667
YAPC,NaN,NaN,0.095444,NaN,NaN,0.302556,NaN,NaN,0.048444,NaN,...,0.482000,0.077778,NaN,0.275556,0.069000,NaN,0.333000,NaN,0.104222,0.066111
YMB1E,NaN,NaN,0.124333,NaN,NaN,0.374778,NaN,NaN,0.251778,NaN,...,0.607556,0.115222,NaN,0.310111,0.140000,0.620667,0.516222,NaN,0.190889,0.258444
ZR751,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.220778,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [83]:
sorted( gcsi_info_sel['drugid'] )

['5-Fluorouracil',
 'AZ-628',
 'AZD7762',
 'AZD8055',
 'Azacitidine',
 'Bortezomib',
 'Bosutinib',
 'Chir-99021',
 'Cisplatin',
 'Crizotinib',
 'Dabrafenib',
 'Dasatinib',
 'Docetaxel',
 'Erastin',
 'Erlotinib',
 'Etoposide',
 'GSK2636771',
 'Gefitinib',
 'Gemcitabine',
 'Ibrutinib',
 'Idelalisib',
 'JQ1 compound',
 'Lapatinib',
 'Lenalidomide',
 'MI-2',
 'MK-2206',
 'MLN2480',
 'Methotrexate',
 'NU-7441',
 'Narciclasine',
 'Nilotinib',
 'Oligomycin A',
 'PF-4708671',
 'PLX4720',
 'Paclitaxel',
 'Palbociclib',
 'Pictilisib',
 'TGX221',
 'Tanespimycin',
 'Tivantinib',
 'Vincaleukoblastine',
 'Vincristine',
 'Vorinostat']

In [84]:
dict_for_replace={'5-FU':'5-Fluorouracil','CHIR-99021':'Chir-99021',"JQ1":'JQ1 compound',
                    "NU7441":'NU-7441','PLX-4720':'PLX4720','TGX-221':'TGX221',
                  'AZD-7762':'AZD7762','AZD-8055':'AZD8055','Vinblastine':'Vincaleukoblastine',
                    'OligomycinA':'Oligomycin A','17-AAG':'Tanespimycin'}

In [85]:
gcsi_add=gcsi_add.rename(columns=dict_for_replace)

In [86]:
dict_name_cid={k:str(int(v)) for k,v in zip(gcsi_info_sel['drugid'],gcsi_info_sel['cid'])}

In [87]:
gcsi_add=gcsi_add.loc[:,gcsi_add.columns.isin(dict_name_cid)]

In [88]:
gcsi_add.columns=['gCSI_'+x+"_CID"+dict_name_cid[x] for x in gcsi_add.columns]
gcsi_add

,gCSI_Tanespimycin_CID6505803,gCSI_5-Fluorouracil_CID3385,gCSI_AZ-628_CID11676786,gCSI_AZD7762_CID11152667,gCSI_AZD8055_CID25262965,gCSI_Azacitidine_CID9444,gCSI_Bortezomib_CID387447,gCSI_Bosutinib_CID5328940,gCSI_Chir-99021_CID9956119,gCSI_Cisplatin_CID24191118,...,gCSI_Oligomycin A_CID5472285,gCSI_PF-4708671_CID51371303,gCSI_PLX4720_CID24180719,gCSI_Paclitaxel_CID36314,gCSI_Palbociclib_CID5330286,gCSI_TGX221_CID9907093,gCSI_Tivantinib_CID11494412,gCSI_Vincaleukoblastine_CID241902,gCSI_Vincristine_CID5978,gCSI_Vorinostat_CID5311
22RV1,NaN,0.279667,0.166333,NaN,NaN,NaN,NaN,NaN,NaN,0.428333,...,NaN,0.014222,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
769P,NaN,NaN,NaN,NaN,NaN,0.171000,NaN,NaN,NaN,NaN,...,0.246111,NaN,NaN,0.437667,0.064444,0.156000,0.413333,0.152444,-0.034000,0.140556
8305C,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.574000,NaN,NaN,0.652556,NaN,NaN,0.732333,NaN,0.159778,0.465000
8505C,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.251444,NaN,NaN,0.558333,NaN,NaN,0.391000,NaN,0.143000,0.340333
A253,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
WSUDLCL2,NaN,NaN,NaN,NaN,NaN,0.345889,NaN,NaN,NaN,NaN,...,0.124222,NaN,NaN,1.016444,0.132333,0.062889,0.674444,0.465000,0.568667,0.777667
YAPC,NaN,NaN,0.095444,NaN,NaN,0.302556,NaN,NaN,NaN,0.082000,...,0.482000,0.077778,NaN,0.275556,0.069000,NaN,0.333000,NaN,0.104222,0.066111
YMB1E,NaN,NaN,0.124333,NaN,NaN,0.374778,NaN,NaN,NaN,0.045778,...,0.607556,0.115222,NaN,0.310111,0.140000,0.620667,0.516222,NaN,0.190889,0.258444
ZR751,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.053333,...,NaN,0.220778,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Write out

In [42]:
print(gdsc1_add.shape)
print(gdsc2_add.shape)
print(ctrp1_auc_sel.T.shape)
print(ctrp2_auc_sel.T.shape)
print(prism_auc.T.shape)
print(gcsi_add.shape)

(987, 240)
(809, 166)
(242, 90)
(887, 173)
(481, 172)
(569, 41)


In [89]:
gdsc1_add.to_csv('../data/GDSC1_AUC.txt',sep='\t')
gdsc2_add.to_csv('../data/GDSC2_AUC.txt',sep='\t')
ctrp1_auc_sel.T.to_csv('../data/CTRP1_AUC.txt',sep='\t')
ctrp2_auc_sel.T.to_csv('../data/CTRP2_AUC.txt',sep='\t')
prism_auc.T.to_csv('../data/PRISM_AUC.txt',sep='\t')
gcsi_add.to_csv('../data/gCSI_AOC.txt',sep='\t')